Following the notebook: http://nlp.seas.harvard.edu/2018/04/03/attention.html

See also the following two blog posts:

[1] https://huggingface.co/blog/encoder-decoder?utm_campaign=Hugging%2BFace&utm_medium=web&utm_source=Hugging_Face_3

[2] http://jalammar.github.io/illustrated-transformer/


In [1]:
import copy
import math
import numpy as np
import time

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import matplotlib.pyplot as plt
import seaborn as sbn
sbn.set_context(context='talk')
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
torch.__version__

'1.4.0'

In [5]:
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# Part I. Model

# 1. Masking

if the original sentence is [1, 2, 3, 4]
  the sequence used as input would be: [1, 2, 3]
  with the following being the target: [2, 3, 4]
  
  In other words, 
      - given [1], the decoder should output 2
      - givne [1, 2], the decoder should output 3
      - given [1, 2, 3], the decoder should output 4


In [6]:
def subsequent_mask(size):
    """
    generate mask for the target sentence
    
    params:
      size: int
      
    return:
      mask: shape (1, size, size)
    """
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [7]:
subsequent_mask(3)[0]

tensor([[ True, False, False],
        [ True,  True, False],
        [ True,  True,  True]])

# 2. Batching

In [8]:
class Batch:
    """
    Object for holding a batch of data with mask 
    """
    def __init__(self, src, trg=None, pad=0):
        """
        params:
          src: source sentence, tensor, shape (batch, src_len)
          trg: target sentence, tensor, shape (batch, trg_len)
                  
        """
        self.src = src  # shape (batch, src_len)
        self.src_mask = (src != pad).unsqueeze(-2)  # shape (batch, 1, src_len)
        if trg is not None:
            self.trg = trg[:, :-1]  # shape (batch, trg_len1), where trg_len = trg_len - 1
            self.trg_y = trg[:, 1:] # sape (batch, trg_len1)
            self.trg_mask = self.make_std_mask(self.trg, pad)  # shape (batch, len1, len1)
            self.ntokens = (self.trg_y != pad).data.sum()
            
    @staticmethod        
    def make_std_mask(tgt, pad):
        """
        params:
          tgt: target sentence, tensor, shape (batch, len)
          pad: padding index, int
        
        return
          mask: shape (batch, len, len)
        """
        padding_mask = (tgt != pad).unsqueeze(-2)  # shape (batch, 1, len)
        future_mask = Variable(subsequent_mask(tgt.size(-1))).type_as(padding_mask.data)  # shape (1, len, len)
        mask = padding_mask & future_mask  # shape (batch, len, len)
        return mask   

# 3. Synthetic Data

In [9]:
def data_generator(V, batch, nbatches):
    """
    use 0 as the index for BOS, beginning of the sentence 
        V-1 as the index for EOS, end of a sentence
    """
    for i in range(nbatches):
        data = np.random.randint(1, V-1, size=(batch, 10))  # 0 is included, V-1 is *not* included
        
        bos = np.zeros(shape=(batch, 1))
        eos = np.ones(shape=(batch, 1)) * (V - 1)
        
        src = Variable(
            torch.from_numpy(np.concatenate([data, eos], axis=1)), 
            requires_grad=False
        ).type(torch.long)

        tgt = Variable(
            torch.from_numpy(np.concatenate([bos, data, eos], axis=1)), 
            requires_grad=False
        ).type(torch.long)
        
        yield Batch(src, tgt, -1)    

# 4. Transformer architecture

## 4.1 Five submodules

### 4.1.1 Positional encoding

In [10]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        # Question: What's the difference between Parameter and Variable?
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

### 4.1.2 Embedding layer

In [11]:
class Embeddings(nn.Module):
    """
    Implement the embedding layer in the Transformer
    """
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.d_model = d_model
        self.lut = nn.Embedding(num_embeddings=vocab, embedding_dim=d_model)
        
    def forward(self, x):
        """
        params:
            x: tokenized input, shape: (batch, sent_len)
        returns:
            x: embeddings of input, shape: (batch, sent_len, d_model)
        """
        return self.lut(x) * math.sqrt(self.d_model)    

### 4.1.3 Layer normalization

In [12]:
class LayerNorm(nn.Module):
    """
    Implement layer normalization
    """
    def __init__(self, features, eps=1e-6):
        """
        params:
            features: no. of features, d_model in practice, int            
        """
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
    
    def forward(self, x):
        """
        params:
            x: tensor, shape: (batch, sent_len, d_model)
        return:
            x: Layer-normed x, tensor, shape: (batch, sent_len, d_model)
        """
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

### 4.1.4 Sublayer connection

In [13]:
class SublayerConnection(nn.Module):
    """
    Implement a sublayer connection in an Encoder/Decoder layer
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, sublayer):
        """
        params:
            x: tensor, shape: (batch, sent_len, d_model)
            layer: an nn.Moduel representing a layer before the sublayer connection
        """
        # Question: this seems to be different from what's specified in the paper
        return x + self.dropout(sublayer(self.norm(x)))

### 4.1.5 Position-wise feed-forward layer

In [14]:
class PositionwiseFeedForward(nn.Module):
    """
    Implement the position-wise feed-forward sublayer inside an Encoder/Decoder layer
    """
    def __init__(self, d_model, d_ff, dropout):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        params:
            x: output from the attention layer, shape: (batch, sent_len, d_model)
        returns:
            x: processed x, shape: (batch, sent_len, d_model)
        """
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

## 4.2 EncoderDecoder wrapper

In [15]:
class EncoderDecoder(nn.Module):
    
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
      
    def encode(self, src, src_mask):
        """
        params:
            src, torch.tensor, shape: (batch, sent_len)
            src_mask, torch.tensor, shape: (batch, sent_len)
            
        returns:
            encoded source sentences, torch.tensor, shape: (batch, sent_len, d_model)
        """
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        """
        params:
            memory: torch.tensor, shape (batch, sent_len, d_model)
            src_mask: torch.tensor, shape (batch, sent_len)
            tgt: torch.tensor, shape (batch, sent_len1)
            tgt_mask: torch.tensor, shape (batch, sent_len1, sent_len1)
            
        returns:
            decoded target sentences in the embedding space, torch.tensor, shape: (batch, sent_len1, d_model)        
        """
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)   

    def forward(self, src, tgt, src_mask, tgt_mask):
        """
        params:
            src: torch.tensor, shape: (batch, sent_len)
            src_mask: torch.tensor, shape: (batch, sent_len)
            tgt: torch.tensor, shape: (batch, sent_len1)
            tgt_mask: torch.tensor, shape (batch, sent_len1, sent_len1)
        """
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)                

## 4.3 Generator

In [16]:
class Generator(nn.Module):
    
    def __init__(self, d_model: int, vocab: int):
        """
        params:
            d_model, int, dimension
            vocab, int, vocabulary size
        """
        super(Generator, self).__init__()
        self.d_model = d_model
        self.vocab = vocab
        self.proj = nn.Linear(in_features=d_model, out_features=vocab)
        
    def forward(self, x):
        """
        params:
            x: decoded target sentence in the embedding space, shape: (batch, sent_len1, d_model)
        returns:
            log softmax probabilities, shape: (batch, sent_len1, vocab)
        """
        return F.log_softmax(self.proj(x), dim=-1)

## 4.4 Encoder

In [17]:
class Encoder(nn.Module):
    """
    Implement the encoder in the Transformer
    """
    def __init__(self, layer, N):
        """
        layer: an EncoderLayer object representing one encoder layer in the Transformer
        N: num of repeats, int
        """
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, mask):
        """
        params:
            x: embddings of input, shape: (batch, sent_len, d_model)
            mask: mask of tokens, shape: (batch, 1, sent_len)
        returns:
            encoded input, shape: (batch, sent_len, d_model)
        """
        for layer in self.layers:
            x = layer(x, mask)
        # Question: the original paper doesn't seem to mention layer norm as the final step
        return self.norm(x)        

In [18]:
class EncoderLayer(nn.Module):
    """
    Implement one encoder layer in the Transformer
    """
    def __init__(self, size, self_attn, feed_forward, dropout):
        """
        params:
            size: dimenion of the input, int
            self_attn: self-attention layer, an nn.Module
            feed_forward: a feed_forward layer, an nn.Module
            dropout: dropout percentage, float
        """
        super(EncoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        
    def forward(self, x, mask):
        """
        params:
            x: embddings of input, shape: (batch, sent_len, d_model)
            mask: mask of tokens, shape: (batch, 1, sent_len)
        """
        x = self.sublayer[0](x, lambda x1: self.self_attn(x1, x1, x1, mask))
        return self.sublayer[1](x, self.feed_forward)

## 4.5 Decoder

In [19]:
class Decoder(nn.Module):
    """
    Implement the decoder in the Transformer
    """
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, y, memory, src_mask, tgt_mask):
        """
        params:
            y: embeddings of target sentences, shape: (batch, sent_len1, d_model)
            memory: encoded source sentences, shape: (batch, sent_len, d_model)
            src_mask: source sentences mask, shape: (batch, 1, sent_len)
            tgt_mask: target sentences mask, shape: (batch, sent_len1, sent_len1)
        returns:
            decoded target sentences, shape: (batch, sent_len1, d_model)
        """
        for layer in self.layers:
            y = layer(y, memory, src_mask, tgt_mask)
        # Question: the original paper doesn't seem to mention layer norm as the final step            
        return self.norm(y)

In [20]:
class DecoderLayer(nn.Module):
    """
    Implement a decoder layer in the Transformer
    """
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        """
        self_attn: self-attention layer attending to the target sentence, an nn.Module
        src_attn: cross-attention layer attending to the source sentence, an nn.Module
        feed_forward: position-wise feed-forward layer, an nn.Module
        size: size of the input, int, set to d_model in practice
        dropout: dropout rate used in the sublayer connection, float
        """
        super(DecoderLayer, self).__init__()
        self.size = size        
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayers = clones(SublayerConnection(size, dropout), 3)
        
    def forward(self, y, memory, src_mask, tgt_mask):
        """
        params:
            y: embeddings of target sentences, shape: (batch, sent_len1, d_model)
            memory: encoded source sentences, shape: (batch, sent_len, d_model)
            src_mask: source sentences mask, shape: (batch, 1, sent_len)
            tgt_mask: target sentences mask, shape: (batch, sent_len1, sent_len1)
        returns:
            decoded target sentences, shape: (batch, sent_len1, d_model)
        """
        y = self.sublayers[0](y, lambda y1: self.self_attn(y1, y1, y1, tgt_mask))
        y = self.sublayers[1](y, lambda y1: self.src_attn(y1, memory, memory, src_mask))
        y = self.sublayers[2](y, self.feed_forward)
        return y

## 4.6 Attention

In [21]:
def attention(query, key, value, mask=None, dropout=None):
    """
    params:
        query: tensor, shape: (batch, h, sent_len, d_k)
        key: tensor, shape: (batch, h, sent_len, d_k)
        value: tensor, shape: (batch, h, sent_len, d_k)
    returns:
        x: weighted values based on the attention mechanism, shape: (batch, h, sent_len, d_k)
        p_attn: (batch, h, sent_len, d_k)
        
    General notes:    
        Note 1:
            Q: (N_q, d_k)    
            K: (N_k, d_k)
            V: (N_k, d_v)

            QK^T: (N_q, N_k)
            softmax(QK^T, dim=-1): (N_q, N_k)
            matmul(softmax(QK^T, dim=-1), V): (N_q, d_v)
            
        Note 2:    
            For single-headed attention:
                input: query, key, value: (batch, sent_len, d_model)
                output: x: (batch, sent_len, d_model)

            For multi-headed attention:
                query, key, value: (batch, h, sent_len, d_k), where d_k = d_model // h
                output: (batch, h, sent_len, d_k)            
    """     
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)  # (batch, h, N_q, N_k)
    
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)  # (batch, h, N_q, N_k)
    
    # Question: the original paper did not have this 
    if dropout is not None:
        p_attn = dropout(p_attn)
        
    x = torch.matmul(p_attn, value)  # (batch, h, N_q, d_v)
    return x, p_attn
         

In [22]:
class MultiHeadedAttention(nn.Module):
    """
    Implement a muti-headed attention layer in the transformer
    """
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % 8 == 0
        self.d_k = d_model // h        
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query, key, value, mask=None):
        """
        params:
            query: tensor, shape: (batch, sent_len, d_model)
            key: tensor, shape: (batch, sent_len, d_model)
            value: tensor, shape: (batch, sent_len, d_model)
        returns:
            x: weighted values based on the attention mechanism, shape: (batch, sent_len, d_model)
        """
        if mask is not None:
            # same mask applied to all h heads
            mask = mask.unsqueeze(1)
        
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k
        """
        l: W_q, W_k, or W_v, which is nn.Linear(d_model, h * d_k)
        x: query, key, value: (batch, sent_len, d_model)
        
        l(x): (batch, sent_len, h * d_k)
        l(x).view(batch, -1, h, d_k).transpose(1,2): (batch, h, sent_len, d_k)
        """
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) 
                             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch
        x, self.attn = attention(query, key, value, mask, dropout=self.dropout)
        # x: (batch, h, sent_len, d_k)
        
        # 3) Concate all the heads 
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)
        

# 5. Building a Transformer model

In [23]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    model = EncoderDecoder(
        Encoder(
            EncoderLayer(
                d_model,
                MultiHeadedAttention(h, d_model, dropout),
                PositionwiseFeedForward(d_model, d_ff, dropout),
                dropout                                
            ),
            N
        ),
        Decoder(
            DecoderLayer(
                d_model,
                MultiHeadedAttention(h, d_model),
                MultiHeadedAttention(h, d_model),
                PositionwiseFeedForward(d_model, d_ff, dropout),
                dropout                                
            ),
            N
        ),
        nn.Sequential(Embeddings(d_model, src_vocab), PositionalEncoding(d_model, dropout)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), PositionalEncoding(d_model, dropout)),
        Generator(d_model, tgt_vocab)        
    )
    
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    
    return model


# 6. Training utilities

## 6.1 Training loop

In [24]:
def run_epoch(data_iter, model, loss_compute):
    start = time.time()
    total_loss = 0    
    total_tokens = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)

        total_loss += loss                
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print(f'Epoch step: {i}, Loss: {loss / batch.ntokens}, Tokens per Sec: {tokens / elapsed}')
            tokens = 0            
            start = time.time()
    return total_loss / total_tokens

## 6.2 Optimizer and Scheduler

In [25]:
class NoamOpt:
    """
    Implement the Noam scheduler that wrapper an optimizer
    """
    def __init__(self, model_size, factor, warmup, optimizer):
        self.model_size = model_size
        self.factor = factor
        self.warmup = warmup
        self.optimizer = optimizer
        self._step = 0
        self._rate = 0
        
    def rate(self, step=None):
        if step is None:
            step = self._step
        return self.factor * (self.model_size ** (-0.5) * min(step ** (-0.5), step * self.warmup ** (-1.5)))
    
    def step(self):
        self._step += 1
        lr = self.rate()
        self._rate = lr
        for p in self.optimizer.param_groups:
            p['lr'] = lr
        self.optimizer.step()
        

In [26]:
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
                   torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
                  )

## 6.3 Label smoothing

In [27]:
class LabelSmoothing(nn.Module):
    """
    Implement label smoothing
    """
    def __init__(self, size, padding_idx=0, smoothing=0.0):
        """
        params:
            size: size of the vocabulary, int
            padding_idx: padding index, int
            smoothing: smoothing factor, float
        """
        super(LabelSmoothing, self).__init__()
        self.size = size
        self.padding_idx = padding_idx
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing
        self.criterion = nn.KLDivLoss(size_average=False)
        self.true_dist = None
        
    def forward(self, x, target):
        """
        params:
            x: log-probabilities, tensor, shape: (batch * tgt_sent_len, vocab_size)
            target: indices of the target tokens, tensor, shape: (batch * tgt_sent_len, vocab_len)
        return:
            KL-divergence loss between the model output probabilities and the smoothed true probabilities
        """
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))        

## 6.4 Loss Computation

In [28]:
class SimpleLossCompute:
    """
    A wrapper of generator, KL-divergence criterion with label smoothing, and NoamOpt (optional)
    """
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
    
    def __call__(self, x, y, norm):
        """
        params:
            x: embedded target sentences, tensor, shape: (batch, tgt_sent_len, d_model)
            y: true target sentences, tensor, shape: (batch, tgt_sent_len)
        return:
            total loss, float
        """
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)),
                              y.contiguous().view(-1)
                             ) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.item() * norm

# Part II. Training 

# 1. Train a simple copy task 

## 1.1 training a model

In [29]:
V = 11
criterion = LabelSmoothing(V, -1, smoothing=0.0)  # 0 is the index for BOS, V is the index for EOS
model = make_model(V, V, N=2)
model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400, 
                   torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

for epoch in range(10):
    model.train()
    run_epoch(data_generator(V, 30, 20), model, SimpleLossCompute(model.generator, criterion, model_opt))
    
    model.eval()
    print(run_epoch(data_generator(V, 30, 5), model, SimpleLossCompute(model.generator, criterion, None)))    

Epoch step: 1, Loss: 2.6852035522460938, Tokens per Sec: 771.4193725585938
Epoch step: 1, Loss: 1.6924911737442017, Tokens per Sec: 1001.3639526367188
tensor(1.6462)
Epoch step: 1, Loss: 1.723552942276001, Tokens per Sec: 713.389404296875
Epoch step: 1, Loss: 1.4428123235702515, Tokens per Sec: 856.1842651367188
tensor(1.4456)
Epoch step: 1, Loss: 1.6433860063552856, Tokens per Sec: 761.352783203125
Epoch step: 1, Loss: 1.1810276508331299, Tokens per Sec: 952.20263671875
tensor(1.1986)
Epoch step: 1, Loss: 1.5336004495620728, Tokens per Sec: 742.3085327148438
Epoch step: 1, Loss: 1.0662708282470703, Tokens per Sec: 935.1001586914062
tensor(1.0062)
Epoch step: 1, Loss: 1.1117819547653198, Tokens per Sec: 672.2886352539062
Epoch step: 1, Loss: 0.5911529064178467, Tokens per Sec: 805.4971313476562
tensor(0.5955)
Epoch step: 1, Loss: 0.9078608751296997, Tokens per Sec: 623.6346435546875
Epoch step: 1, Loss: 0.5403550863265991, Tokens per Sec: 841.24560546875
tensor(0.5284)
Epoch step: 1, L

## 1.2 Inference

In [32]:
def greedy_decode(model, src, src_mask, max_len, start_symbol=0):
    """
    params:
        model: a trained EncoderDecoder model
        src: input sentence, shape: (batch, src_sent_len)
        src_mask: mask of the input sentence, shape: (batch, 1, src_sent_len)
        max_len: max output length, int
        start_symbol: index of the BOS token
    return
        decoded sentence, shape: (batch, max_len)
    """
    memory = model.encode(src, src_mask)
    ys = torch.zeros((src.size(0), 1)).fill_(start_symbol).type_as(src.data)
    for i in range(max_len):
        tgt_mask = Batch.make_std_mask(ys, -1)        
        out = model.decode(
            memory, 
            src_mask, 
            Variable(ys), 
            tgt_mask
        ) # (batch, len, d_model)
        #prob = model.generator(out)[:, -1, :]
        prob = model.generator(out[:, -1, :])  # shape: (batch, V)
        _, next_token = torch.max(prob, dim=1)  # shape: (batch, )
        ys = torch.cat([ys, next_token.view(-1, 1)], dim=1) 
    return ys


In [33]:
model.eval()
src = Variable(torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9]]) )
src_mask = Variable(torch.ones(src.size(0), 1, src.size(1)))
print(greedy_decode(model, src, src_mask, 13, 0))

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 9, 8]])


# Appendix: steps to implement label smoothing

In [35]:
V = 5
padding_idx = 1 
smoothing = 0.3
confidence = 1 - smoothing


target = torch.from_numpy(np.array([3, 0, 1, 2, 4, 2, 1]))

x = torch.rand((target.size(0), V))


true_dist = torch.zeros((target.size(0), V))
true_dist

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [36]:
# Assign smoothing probabilities to entries corresponding to "incorrect" tokens
true_dist.fill_(smoothing / (V-2))  # 1 for the correct token, 1 for the padding index
true_dist

tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000]])

In [37]:
# Assign the smoothed probabilities to the true tokens
true_dist.scatter_(1, target.data.unsqueeze(1), confidence)
true_dist

tensor([[0.1000, 0.1000, 0.1000, 0.7000, 0.1000],
        [0.7000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.7000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.7000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.7000],
        [0.1000, 0.1000, 0.7000, 0.1000, 0.1000],
        [0.1000, 0.7000, 0.1000, 0.1000, 0.1000]])

In [38]:
# Zero out the column of the padding index 
true_dist[:, padding_idx] = 0
true_dist

tensor([[0.1000, 0.0000, 0.1000, 0.7000, 0.1000],
        [0.7000, 0.0000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.0000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.0000, 0.7000, 0.1000, 0.1000],
        [0.1000, 0.0000, 0.1000, 0.1000, 0.7000],
        [0.1000, 0.0000, 0.7000, 0.1000, 0.1000],
        [0.1000, 0.0000, 0.1000, 0.1000, 0.1000]])

In [39]:
# Zero out the rows corresponding to the padding tokens
mask = torch.nonzero(target.data == padding_idx)
mask

tensor([[2],
        [6]])

In [40]:
mask.squeeze()

tensor([2, 6])

In [41]:
if mask.dim() > 0:
    print(mask.dim())
    true_dist.index_fill_(0, mask.squeeze(), 0.0)
true_dist    

2


tensor([[0.1000, 0.0000, 0.1000, 0.7000, 0.1000],
        [0.7000, 0.0000, 0.1000, 0.1000, 0.1000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1000, 0.0000, 0.7000, 0.1000, 0.1000],
        [0.1000, 0.0000, 0.1000, 0.1000, 0.7000],
        [0.1000, 0.0000, 0.7000, 0.1000, 0.1000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [42]:
# Noticce that when calculating KL divergence using nn.KLDivLoss, the zero-entries will be skipped 
# 
# The following two are equivalent
# criterion = nn.KLDivLoss(size_average=False)
criterion = nn.KLDivLoss(reduction='sum')

In [43]:
criterion(x, Variable(true_dist, requires_grad=False))

tensor(-7.7822)